<a href="https://colab.research.google.com/github/Orneyfish/Orney/blob/master/BERT_SENT_EMB_SEARCH_Custom_title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install bert-serving-server bert-serving-client
! pip install tensorflow-gpu==1.14.0-rc0

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.14.0-rc0 (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 1.3.0, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.9.0, 1.10.0, 1.10.1, 1.11.0, 1.12.0, 1.12.2, 1.12.3, 1.13.1, 1.13.2, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3)
ERROR: No matching distribution found for tensorflow-gpu==1.14.0-rc0


In [0]:
! pip install wget

In [65]:
# Install the library using pip
!pip install sentence-transformers

In [0]:
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [89]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/title.csv", delimiter='\t', header=None, names=['title', 'abstract', 'body_text'] )

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 12



,title,abstract,body_text
5,Journal Pre-proof What Should Gastroenterologi...,NaN,NaN
11,Does genetic diversity limit disease spread in...,NaN,NaN
3,The Impact of Host-Based Early Warning on Dise...,NaN,NaN
10,When are pathogen genome sequences informative...,NaN,NaN
8,viruses Can Bats Serve as Reservoirs for Arbov...,NaN,NaN
7,Dengue Virus Glycosylation: What Do We Know?,NaN,NaN
4,Allergologia et immunopathologia Does atopy af...,NaN,NaN
6,Pacific Observatory on Health Systems and Poli...,NaN,NaN
2,Comment 760 www,NaN,NaN
1,Systematic review Does reduced MHC diversity d...,NaN,NaN


In [90]:
df.sample(5)[['title', 'body_text']]

,title,body_text
3,The Impact of Host-Based Early Warning on Dise...,NaN
0,title,NaN
8,viruses Can Bats Serve as Reservoirs for Arbov...,NaN
2,Comment 760 www,NaN
10,When are pathogen genome sequences informative...,NaN


In [0]:
# Get the lists of sentences and their labels.
sentences = df.title.values

In [92]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [93]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  title
Tokenized:  ['title']
Token IDs:  [2516]


In [94]:
# A corpus is a list with documents split by sentences.

# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector - note includes negative values [ 1.15534894e-01 -3.01038295e-01  2.28389716e+00  8.43254983e-01
  5.70622921e-01 -3.94892693e-02 -4.96040732e-01  6.58538640e-01
  7.37184957e-02  2.53860980e-01 -9.87959683e-01  1.28782347e-01
  3.27380925e-01  9.03856695e-01  9.40645695e-01 -2.32353628e-01
 -7.33541965e-01  1.71956062e-01  2.28647396e-01 -1.06962144e+00
 -2.43077517e-01  4.92174625e-01  2.93649584e-01 -7.91571617e-01
  2.32278958e-01 -6.35309517e-01  1.73399597e-02 -2.23125911e+00
 -9.74390507e-01 -2.06972525e-01 -1.57009512e-01 -8.45554173e-01
  3.28928739e-01  5.80681324e-01  4.10945892e-01  2.51200140e-01
 -7.88593292e-01  2.28311896e-01 -1.75099373e-01 -4.62637097e-01
  1.63711512e+00  1.64804652e-01  1.01361132e+00 -2.05736291e-02
 -5.98165452e-01 -2.36950994e-01 -2.27621675e-01 -3.79146844e-01
  1.27082393e-01 -1.01565158e+00 -7.50274241e-01 -5.68745434e-01
 -3.15868296e-02  3.78211528e-01 -6.33806810e-02  7.

In [95]:
!pip install sklearn
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [98]:
query = 'COVID-19 is pandemic' #@param {type: 'string'}

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest sentences of the corpus for each query sentence based on cosine similarity
number_top_matches =  2#@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: COVID-19 is pandemic

Top 5 most similar sentences in corpus:
The Impact of Host-Based Early Warning on Disease Outbreaks (Cosine Score: 0.5847)
Journal Pre-proof What Should Gastroenterologists and Patients Know About COVID-19? (Cosine Score: 0.5723)
